In [ ]:
import time

import numpy as np

from mc_lab.inverse_transform import (
    StratifiedInverseTransformSampler,
    create_sampler,
)

In [ ]:
# Demonstration and examples
def demo_unified_framework():
    """Demonstrate the unified framework with different samplers."""

    print("Unified Inverse Transform Sampling Framework Demo")
    print("=" * 60)

    # Example 1: Analytical inverse CDF (Exponential distribution)
    print("\n1. Analytical Method - Exponential Distribution")
    print("-" * 50)

    def exp_inverse_cdf(u, rate=2.0):
        return -np.log(1 - u) / rate

    # Create analytical sampler
    exp_sampler = create_sampler(
        inverse_cdf=lambda u: exp_inverse_cdf(u, rate=2.0),
        method="analytical",
        random_state=42,
    )

    samples = exp_sampler.sample(100000)
    print(f"Sample mean: {np.mean(samples):.4f} (expected: {0.5:.4f})")
    print(f"Sample std:  {np.std(samples):.4f} (expected: {0.5:.4f})")

    # Example 2: Numerical inverse CDF (Beta distribution)
    print("\n2. Numerical Method - Beta Distribution")
    print("-" * 50)

    from scipy.stats import beta

    alpha, beta_param = 2, 5

    def beta_cdf(x):
        return beta.cdf(x, alpha, beta_param)

    # Create numerical sampler
    beta_sampler = create_sampler(
        cdf=beta_cdf, x_range=(0, 1), method="numerical", n_points=1000, random_state=42
    )

    samples = beta_sampler.sample(10000)
    expected_mean = alpha / (alpha + beta_param)
    print(f"Sample mean: {np.mean(samples):.4f} (expected: {expected_mean:.4f})")

    # Example 3: Enhanced samplers using the same base
    print("\n3. Enhanced Sampling Methods")
    print("-" * 50)

    # Stratified sampling
    stratified_sampler = StratifiedInverseTransformSampler(
        base_sampler=beta_sampler, n_strata=20, random_state=42
    )

    stratified_samples = stratified_sampler.sample(10000)
    print(f"Stratified mean: {np.mean(stratified_samples):.4f}")
    print(f"Variance reduction: {np.var(samples) / np.var(stratified_samples):.2f}x")

    # Example 4: Quantile sampling
    print("\n4. Quantile Sampling")
    print("-" * 50)

    quantiles = np.array([0.1, 0.25, 0.5, 0.75, 0.9])
    quantile_values = beta_sampler.sample_quantiles(quantiles)

    print("Beta(2,5) Quantiles:")
    for q, val in zip(quantiles, np.asarray(quantile_values)):
        print(f"  Q({q:.2f}) = {val:.4f}")

    # Performance comparison
    print("\n5. Performance Comparison")
    print("-" * 50)

    n_samples = 50000

    methods = [
        ("Analytical", exp_sampler),
        ("Numerical", beta_sampler),
        ("Stratified", stratified_sampler),
    ]

    for name, sampler in methods:
        start_time = time.time()
        _ = sampler.sample(n_samples, random_state=42)
        elapsed = time.time() - start_time
        print(f"{name:<12}: {elapsed:.4f}s ({n_samples / elapsed:,.0f} samples/s)")


if __name__ == "__main__":
    demo_unified_framework()

Unified Inverse Transform Sampling Framework Demo

1. Analytical Method - Exponential Distribution
--------------------------------------------------
Sample mean: 0.4980 (expected: 0.5000)
Sample std:  0.4965 (expected: 0.5000)

2. Numerical Method - Beta Distribution
--------------------------------------------------
Sample mean: 0.2823 (expected: 0.2857)

3. Enhanced Sampling Methods
--------------------------------------------------
Stratified mean: 0.2854
Variance reduction: 0.99x
Quasi-random mean: 0.2857

4. Quantile Sampling
--------------------------------------------------
Beta(2,5) Quantiles:
  Q(0.10) = 0.0926
  Q(0.25) = 0.1612
  Q(0.50) = 0.2645
  Q(0.75) = 0.3895
  Q(0.90) = 0.5103

5. Performance Comparison
--------------------------------------------------
Analytical  : 0.0006s (81,506,102 samples/s)
Numerical   : 0.0028s (17,600,940 samples/s)
Stratified  : 0.0038s (13,283,202 samples/s)
Quasi-random: 0.0027s (18,306,145 samples/s)


/home/carstenj/dev/mc-lab/src/mc_lab/inverse_transform.py:395: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  return sampler.random(n).flatten()
